In [6]:
import pandas as pd
import numpy as np
import pickle

accidents = pd.read_pickle("final_accidents2.pickle")
drivers = pd.read_pickle("final_drivers.pickle")

from pyproj import Proj, transform

from_proj = Proj(init="epsg:4326")
to_proj = Proj(init="epsg:3857")

def convert(longitudes, latitudes):
    """Converts latlon coordinates to meters.
    Inputs:
    longitudes (array-like) : array of longitudes
    latitudes (array-like) : array of latitudes
    Example:
    x,y = convert(accidents.LONGITUD, accidents.LATITUDE)
    """
    x_vals = []
    y_vals = []
    for lon, lat in zip(longitudes, latitudes):
        x, y = transform(from_proj, to_proj, lon, lat)
        x_vals.append(x)
        y_vals.append(y)
    return x_vals, y_vals

accidents['LONGITUD'], accidents['LATITUDE'] = convert(accidents['LONGITUD'], accidents['LATITUDE'])

In [7]:
accidents.head()

,ST_CASE,STATE,LATITUDE,LONGITUD,HOUR,DAY,MONTH,YEAR,DRUNK_DR,WEATHER,SP,FATALS
0,10001,1,3.847758e+06,-9.501641e+06,4,15,1,2010,1,1,0,1
1,10002,1,3.688778e+06,-9.679975e+06,6,11,1,2010,0,1,0,1
2,10003,1,3.592766e+06,-9.805664e+06,15,14,1,2010,0,1,1,1
3,10004,1,4.011185e+06,-9.605888e+06,1,21,1,2010,0,2,0,1
4,10005,1,3.936490e+06,-9.661109e+06,6,4,1,2010,0,1,0,1


In [8]:
drivers.head()

,ST_CASE,VEH_NO,AGE,DRINKING,SPEEDREL,YEAR
0,10001,1,51,9,0,2010
2,10002,1,44,0,0,2010
3,10003,1,27,9,1,2010
4,10003,2,45,0,0,2010
5,10003,3,28,0,0,2010


In [9]:
from bokeh.plotting import figure, output_file, show, Figure
from bokeh.models import WMTSTileSource, ColumnDataSource

fig = Figure(plot_width=1100, plot_height=650,
x_range=(-13000000, -7000000), y_range=(2750000, 6250000),tools=["wheel_zoom", "pan"], active_scroll="wheel_zoom",webgl = True )
fig.axis.visible = False
STAMEN_TONER_BACKGROUND = WMTSTileSource(url='http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png', attribution=(
'Map tiles by <a href="http://stamen.com">Stamen Design</a>, '
'under <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a>.'
'Data by <a href="http://openstreetmap.org">OpenStreetMap</a>, '
'under <a href="http://www.openstreetmap.org/copyright">ODbL</a>'))

fig.add_tile(STAMEN_TONER_BACKGROUND)

In [10]:
# Three group: Drunk, speeding, and neither

d = accidents[accidents["DRUNK_DR"]==1]
s = accidents[accidents["SP"]==1]
o = accidents[(accidents["SP"]==0)&(accidents['DRUNK_DR']==0)]

# Plot drunk group
d1 = pd.DataFrame({"x_vals":d["LONGITUD"], "y_vals":d["LATITUDE"]})
d1["fill_alpha"] = 1
d1["fill_color"] = "red"
d1["line_alpha"] = 0
d1["size"] = 2

cir_source = ColumnDataSource(d1)
cir = fig.circle(x="x_vals", y="y_vals", source=cir_source, radius="size", fill_color="fill_color", fill_alpha="fill_alpha", line_width=0)

# Plot neither group
o1 = pd.DataFrame({"x_vals":o["LONGITUD"], "y_vals":o["LATITUDE"]})
o1["fill_alpha"] = 1
o1["fill_color"] = "green"
o1["line_alpha"] = 0
o1["size"] = 2

cir_source2 = ColumnDataSource(o1)
cir2 = fig.circle(x="x_vals", y="y_vals", source=cir_source2, radius="size", fill_color="fill_color", fill_alpha="fill_alpha", line_width=0)

# Plot speeding group
s1 = pd.DataFrame({"x_vals":s["LONGITUD"], "y_vals":s["LATITUDE"]})
s1["fill_alpha"] = 1
s1["fill_color"] = "blue"
s1["line_alpha"] = 0
s1["size"] = 2

cir_source3 = ColumnDataSource(s1)
cir3 = fig.circle(x="x_vals", y="y_vals", source=cir_source3, radius="size", fill_color="fill_color", fill_alpha="fill_alpha",line_width=0)

# Display map
show(fig)